In [ ]:
import os
import time
import requests
import hashlib
import pandas as pd
from IPython.display import display
from selenium import webdriver
from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
# from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service

In [10]:
# .\venv\Scripts\activate
# deactivate
# working click version

def download_images_from_yandex(query, num_images):
    if not os.path.exists('data.csv'):
        # Creating an empty DataFrame from the lists
        columns = ['Name', 'Path', 'Url']
        df = pd.DataFrame(columns=columns)
        print('Created empty file data.csv')
    else:
        df = pd.read_csv('data.csv')

    # Create a base folder named 'data'
    base_folder = 'data'
    if not os.path.exists(base_folder):
        os.makedirs(base_folder)

    # Create a subfolder for the specific query inside 'data'
    # folder_name = os.path.join(base_folder, query.replace(" ", "_"))
    
    # if not os.path.exists(folder_name):
    #     os.makedirs(folder_name)

    # Set Chrome options
    chrome_options = Options()
    # Uncomment the following line for headless mode
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--window-size=1920,1080")

    # Initialize the WebDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.get('https://yandex.ru/images/')

    # Find the search box and perform the search
    search_box = driver.find_element(By.NAME, "text")
    search_box.send_keys(query)
    search_box.submit()

    # Wait for images to load with a longer timeout
    try:
        WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'img.cbir-intent__thumb')))
    except Exception as e:
        print(f"Error during waiting for images: {e}")
        driver.quit()
        return

    # Allow for scrolling to load more images
    scrolls_needed = 5  # Number of scrolls to attempt
    for _ in range(scrolls_needed):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)  # Adjust as necessary for loading time

    # Download images by clicking on them
    count = 0
    while count < num_images:
        # Fetch the image elements again before processing
        image_elements = driver.find_elements(By.CSS_SELECTOR, 'img.ContentImage-Image.ContentImage-Image_clickable')

        if not image_elements:  # If no more images are available, break
            print("No more images found.")
            break
        
        try:
            # Click the image to open a larger version
            img = image_elements[count]  # Get the current image to click
            img_name = img.get_attribute('src')
            hashed_name = hashlib.sha256(img_name.encode('utf-8')).hexdigest()
            # print(hashed_name)
            
            jpg_file_path = os.path.join(base_folder, f"{hashed_name}.jpg")
            png_file_path = os.path.join(base_folder, f"{hashed_name}.png")
            # print(png_file_path)
            # check if images have already been downloaded
            if not os.path.exists(jpg_file_path) and not os.path.exists(png_file_path):
                driver.execute_script("arguments[0].click();", img)
                # img.click()
                time.sleep(2)  # Wait for the larger image to load
                
                # Find the larger image element
                large_image = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'img.MMImage-Origin'))
                )
                img_url = large_image.get_attribute('src') 
                if img_url:
                    img_data = requests.get(img_url).content
                    ext = 'jpg' if 'jpeg' in img_url else 'png'
                    file_path = os.path.join(base_folder, f'{hashed_name}.{ext}')
                    if df[(df['Name'] == hashed_name)].empty:
                        new_row = pd.DataFrame({'Name': [hashed_name], 'Path': [file_path], 'Url': [img_url]})
                        # print(f"Added new row: {new_row['Name']}")
                        df = pd.concat([df, new_row], ignore_index=True)
                    with open(file_path, 'wb') as f:
                        f.write(img_data)
                        print(f"Downloaded {file_path} from URL.")
                        count += 1
                # Close the enlarged image to return to the search results
                driver.back()
                time.sleep(2)  # Allow time for the previous page to load again
            else:
                print(f"image {hashed_name} has been already downloaded.") 
                count += 1
                
        except Exception as e:
            print(f"Error clicking on image or downloading: {e}")
            # Attempt to go back if there was an error
            try:
                driver.back()
                time.sleep(2)  # Allow time for the page to load again
            except Exception as inner_e:
                print(f"Error navigating back: {inner_e}")
    display(df)
    df.to_csv('data.csv', index=False)
    driver.quit()
    print(f"Downloaded {count} images for query '{query}'.")

# Usage
download_images_from_yandex("паспорт рф", 15)  # Adjust the keyword as needed


Created empty file data.csv
Downloaded data\89c8bfe6af38f668815f6c34080b038e5e93c0e845197d46e8ad259b6c6208ff.png from URL.
Downloaded data\fc3a20c81a82f70f99578071f68ac26bbf93c4bbbd995bc6b2eb617f03ba9063.png from URL.
Downloaded data\f41cbf4a0ea62bf66321c881fa0483808dc53cefef9d45ebe8cfee6ea69d2322.png from URL.
Downloaded data\cafdcf7a147a9e529a2161e8a7069b465e1c96d284e8cb4aee1ca57299e40655.png from URL.
Downloaded data\71918b1385fbbd8d70a9d43cbb5854460b9a06759f7929d03241f62fe039242c.png from URL.
Downloaded data\6a608723cf0b7774a788c860867710d0932733a8df3b0cd16ceb7ee95eafa56f.png from URL.
Downloaded data\c070299b386ecd2de9c178275992b2938dabc8495716de24660fff9d5f5aa9a3.png from URL.
Downloaded data\d1a58b4c83ac7eb26636794e6f09b1e4a1137cded0ed526e2b0eebd76bcf40d0.png from URL.
Downloaded data\2061c094b2cd0387cf19af16864fa0d70eebe397719d4ab023740b4ececc0f9b.png from URL.
Downloaded data\d4a3532215f3c46ac5dd982ed603874d6f38749d1ebf76c830165e46139490b4.png from URL.
Downloaded data\035502

,Name,Path,Url
0,89c8bfe6af38f668815f6c34080b038e5e93c0e845197d...,data\89c8bfe6af38f668815f6c34080b038e5e93c0e84...,https://res.cloudinary.com/dbb6h31sb/image/upl...
1,fc3a20c81a82f70f99578071f68ac26bbf93c4bbbd995b...,data\fc3a20c81a82f70f99578071f68ac26bbf93c4bbb...,https://avatars.mds.yandex.net/i?id=a5adb7c517...
2,f41cbf4a0ea62bf66321c881fa0483808dc53cefef9d45...,data\f41cbf4a0ea62bf66321c881fa0483808dc53cefe...,https://avatars.mds.yandex.net/i?id=a205555f68...
3,cafdcf7a147a9e529a2161e8a7069b465e1c96d284e8cb...,data\cafdcf7a147a9e529a2161e8a7069b465e1c96d28...,https://avatars.mds.yandex.net/i?id=804b13354e...
4,71918b1385fbbd8d70a9d43cbb5854460b9a06759f7929...,data\71918b1385fbbd8d70a9d43cbb5854460b9a06759...,https://avatars.mds.yandex.net/i?id=e536524296...
5,6a608723cf0b7774a788c860867710d0932733a8df3b0c...,data\6a608723cf0b7774a788c860867710d0932733a8d...,https://avatars.mds.yandex.net/i?id=c26e5911ba...
6,c070299b386ecd2de9c178275992b2938dabc8495716de...,data\c070299b386ecd2de9c178275992b2938dabc8495...,https://avatars.mds.yandex.net/i?id=e8e1efcdf6...
7,d1a58b4c83ac7eb26636794e6f09b1e4a1137cded0ed52...,data\d1a58b4c83ac7eb26636794e6f09b1e4a1137cded...,https://avatars.mds.yandex.net/i?id=6f4f260811...
8,2061c094b2cd0387cf19af16864fa0d70eebe397719d4a...,data\2061c094b2cd0387cf19af16864fa0d70eebe3977...,https://avatars.mds.yandex.net/i?id=ddfeb88721...
9,d4a3532215f3c46ac5dd982ed603874d6f38749d1ebf76...,data\d4a3532215f3c46ac5dd982ed603874d6f38749d1...,https://steamuserimages-a.akamaihd.net/ugc/184...


Downloaded 15 images for query 'паспорт рф'.


In [ ]:
# working download thumbnails option
# SerpItem
# make clicking work
# locate enlarged image after clicking
# <a href="https://sun9-13.userapi.com/impg/PshPESmU2uloudQfFOtPEI4UnWikwb9M7_cmxw/1z9w2gOUf5s.jpg?size=559x811&amp;quality=96&amp;sign=e7f0537a6dd5994e07a4622cdccbbaa1&amp;type=album" target="_blank" type="button" class="Button2 Button2_pin_circle-brick Button2_size_xl Button2_link Button2_view_default OpenImageButton-Save MMViewerButtons-OpenImage MMViewerButtons-Button" autocomplete="off"><span class="Icon Icon_type_downloadOutline24 Button2-Icon Button2-Icon_side_left" aria-hidden="true"></span><span class="Button2-Text">Открыть</span></a>
# <img class="MMImage-Origin" src="//avatars.mds.yandex.net/i?id=593913c21abff56cc260d5552945e029_l-4097709-images-thumbs&amp;n=13" alt="Picture background" aria-hidden="true">

def download_images_from_yandex(query, num_images):
    # Create a base folder named 'data'
    base_folder = 'data'
    if not os.path.exists(base_folder):
        os.makedirs(base_folder)

    # Create a subfolder for the specific query inside 'data'
    folder_name = os.path.join(base_folder, query.replace(" ", "_"))
    
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Set Chrome options
    chrome_options = Options()
    # Uncomment the following line for headless mode
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--window-size=1920,1080")

    # Initialize the WebDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.get('https://yandex.ru/images/')

    # Find the search box and perform the search
    search_box = driver.find_element(By.NAME, "text")
    search_box.send_keys(query)
    search_box.submit()

    # Wait for images to load with a longer timeout
    try:
        WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'img.cbir-intent__thumb')))
    except Exception as e:
        print(f"Error during waiting for images: {e}")
        driver.quit()
        return

    # Allow for scrolling to load more images
    scrolls_needed = 5  # Number of scrolls to attempt
    for _ in range(scrolls_needed):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)  # Adjust as necessary for loading time

    # Locate image elements on the page after scrolling
    image_elements = driver.find_elements(By.CSS_SELECTOR, 'img.ContentImage-Image.ContentImage-Image_clickable')

    print(f"Found {len(image_elements)} images.")

    # Download images
    count = 0
    for img in image_elements:
        img_url = img.get_attribute('src')
        
        if img_url:
            try:
                img_data = requests.get(img_url).content
                ext = 'jpg' if 'jpeg' in img_url else 'png'
                file_path = os.path.join(folder_name, f'{query}_{count}.{ext}')
                with open(file_path, 'wb') as f:
                    f.write(img_data)
                    print(f"Downloaded {file_path} from URL.")
                count += 1
                if count >= num_images:
                    break
            except Exception as e:
                print(f"Could not download {img_url}: {e}")

    driver.quit()
    print(f"Downloaded {count} images for query '{query}'.")

# Usage
download_images_from_yandex("котята", 5)  # Adjust the keyword as needed


Found 287 images.
Downloaded data\котята\котята_0.png from URL.
Downloaded data\котята\котята_1.png from URL.
Downloaded data\котята\котята_2.png from URL.
Downloaded data\котята\котята_3.png from URL.
Downloaded data\котята\котята_4.png from URL.
Downloaded data\котята\котята_5.png from URL.
Downloaded data\котята\котята_6.png from URL.
Downloaded data\котята\котята_7.png from URL.
Downloaded data\котята\котята_8.png from URL.
Downloaded data\котята\котята_9.png from URL.
Downloaded data\котята\котята_10.png from URL.
Downloaded data\котята\котята_11.png from URL.
Downloaded data\котята\котята_12.png from URL.
Downloaded data\котята\котята_13.png from URL.
Downloaded data\котята\котята_14.png from URL.
Downloaded data\котята\котята_15.png from URL.
Downloaded data\котята\котята_16.png from URL.
Downloaded data\котята\котята_17.png from URL.
Downloaded data\котята\котята_18.png from URL.
Downloaded data\котята\котята_19.png from URL.
Downloaded data\котята\котята_20.png from URL.
Downl

KeyboardInterrupt: 